In [21]:
import numpy as np
import pandas as pd
import pickle

import matplotlib.pyplot as plt
import seaborn as sb


from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score



import keras
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing import image
from keras.wrappers.scikit_learn import KerasClassifier

## Loading the dataframe
Loading the dataframe from the pickle file which was exported from Pre processing stage

In [74]:
with open("data/df.pkl",'rb') as file:
    df = pickle.load(file)

df

Name      GFA     BType              Energy  WeekDay  \
0       Piper Writing Center   4314.0  Academic   1.402795493602753        1   
1       Piper Writing Center   4314.0  Academic  2.6063907146453857        1   
2       Piper Writing Center   4314.0  Academic  0.8264167606830597        1   
3       Piper Writing Center   4314.0  Academic   0.831238254904747        1   
4       Piper Writing Center   4314.0  Academic  4.9978625774383545        1   
...                      ...      ...       ...                 ...      ...   
602573             San Pablo  77484.0   Housing   51.58279991149902        1   
602574             San Pablo  77484.0   Housing   52.42500019073486        1   
602575             San Pablo  77484.0   Housing   50.46094989776611        1   
602576             San Pablo  77484.0   Housing   51.54427433013916        1   
602577             San Pablo  77484.0   Housing   49.68022441864014        1   

        TAVG  TMIN  TMAX  Sem  
0       43.0    38    51    0  
1       43.0    38    51    0  
2       43.0    38    51    0  
3       43.0    38    51    0  
4       43.0    38    51    0  
...      ...   ...   ...  ...  
602573  54.0    45    59    0  
602574  54.0    45    59    0  
602575  54.0    45    59    0  
602576  54.0    45    59    0  
602577  54.0    45    59    0  

[602578 rows x 9 columns]

# Data Understanding and Transformation

In [75]:
df.dtypes

Name        object
GFA        float64
BType       object
Energy      object
WeekDay      int64
TAVG       float64
TMIN         int64
TMAX         int64
Sem          int64
dtype: object

Converting Energy datatype from string to float

In [84]:
df['Energy'] = df['Energy'].apply(lambda x:float(x)).round(1)

#### Checking for negative and missing values for energy

In [72]:
df.loc[df['Energy']<0,['Energy']].count()

Energy    0
dtype: int64

In [73]:
df.loc[df['Energy']<0,['Name']].nunique()

Name    0
dtype: int64

In [66]:
df.isna().sum()

Name           0
GFA        17520
BType      17520
Energy         0
WeekDay        0
TAVG           0
TMIN           0
TMAX           0
Sem            0
dtype: int64

#### Removing the negative and missing values

In [82]:
df = df[df['Energy']>0].dropna()

In [83]:
df['Energy'].describe()

count    571407.000000
mean        109.241787
std         121.975215
min           0.000260
25%          34.878651
50%          66.115526
75%         137.051003
max         757.316742
Name: Energy, dtype: float64

In [ ]:
df['Energy'].apply(lambda x:x)

# Prediction Modeling

## Simple Linear Regression

In [88]:
df.Btype = pd.factorize(df.BType)[0]

In [9]:
featureData = df.drop(['Energy','TMIN','TMAX','Name'],axis=1).values
y = df['Energy'].values

#scaling

scaler = MinMaxScaler(feature_range = (0,1))
X = pd.DataFrame(scaler.fit_transform(featureData))

In [ ]:
#ran the model on EC2 instance for performance

LinReg = LinearRegression(normalize = True)
LinReg.fit(featureData,targetData)

print(LinReg.score(featureData,targetData))

In [8]:
file = open("linear.pkl",'rb')
score = pickle.load(file)

score

0.1330667617808884

## Neural Networks

In [ ]:
#One hot encoding Categorical

BTypedf = pd.get_dummies(df['BType'],prefix='BType')
df = pd.concat([df,BTypedf],axis =1)

In [ ]:
featureData = df.drop(['Energy','Name'],axis=1).values
y = df['Energy'].values

#scaling

scaler = MinMaxScaler(feature_range = (0,1))
X = pd.DataFrame(scaler.fit_transform(featureData))

### Initializing and paramter tuning

In [ ]:
model3 = Sequential()
model3.add(Dense(50,input_dim=59,activation='relu', name = 'layer_1'))
model3.add(Dense(20,activation='relu',name = 'layer_2'))
model3.add(Dense(3,activation='softmax',name = 'output_layer'))

model3.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

In [ ]:
model3.fit(X,y,epochs=100,shuffle=True,verbose=2)

error_rate,accuracy = model3.evaluate(X,y,verbose=0)

print(error_rate, accuracy)

### Evaluating with K fold Cross validation

In [ ]:
#Building the model and evaluating using K fold cross validation

RunName = 'model4'
def createmodel():
    model = Sequential()
    model.add(Dense(10,input_dim=59,activation='relu'))
    modle.add(Dense(5,activation='relu'))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

seed = 7
np.random.seed(seed)

model = KerasClassifier(build_fn = createmodel,epochs = 50, batch_size = 10, verbose = 0)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(model, X, y, cv=kfold)
print(results.mean())